In [ ]:
!apt-get update -y
!apt-get install -y python-xlsxwriter
!pip install openpyxl

In [72]:
import wandb
import pandas as pd
import datetime
import tqdm

username = "memmelma"
project = "final"

filter_name = "scratch"
filter_type = "vo_"
filter_state = "finished"
# filter_state = ""

filter_metric = "Objective/eval_all"
filter_operation = "min"

metrics = ["Objective/eval_forward", "Objective/eval_left", "Objective/eval_right", "eval_all_unified_depth/aux_depth_loss",
           "Objective/train"]

# configs = ["config.VO.MODEL.visual_backbone", "config.VO.MODEL.pretrain_backbone",
#            "config.VO.MODEL.cls_action", "config.VO.TRAIN.depth_aux_loss"]
configs = ["config.VO.MODEL.visual_backbone", "config.VO.MODEL.pretrain_backbone", "config.VO.MODEL.train_backbone", "config.VO.MODEL.visual_type",
           "config.VO.MODEL.cls_action"]

step_threshold = 150

out_name = "exp_vo_scratch"

In [78]:
# retreive runs

run_ids = []
run_names = []
run_ckpts = []

api = wandb.Api(timeout=45)
runs = api.runs(f"{username}/{project}")

runs_df = pd.DataFrame(columns=["name", "id", "state", *configs, filter_metric, *metrics])

for run in tqdm.tqdm(runs):
    
    if filter_state and run._state != filter_state:
        continue
    
    if filter_type and filter_type not in run.name:
        continue
        
    if filter_name and filter_name not in run.name:
        continue
    print(run.name)
    run_dict = {}
    
    # add base attributes
    run_dict["name"] = run.name.split("[")[0]
    run_dict["id"] = run.id 
    run_dict["state"] = run._state
    
    # add config
    for config in configs:
        # traverse config keys
        run_config_tmp = run.config
        for config_iter in config.split('.'):
            run_config_tmp = run_config_tmp[config_iter]
        run_dict[config] = run_config_tmp
    
    # add filter_operation filter_metric
    run_history = run.history()
    select = run_history["_step"]<step_threshold
    
    if filter_operation == "min":
        filter_metric_idx = run_history[filter_metric][select].argmin()
    elif filter_operation == "max":
        filter_metric_idx = run_history[filter_metric][select].argmax()
    else:
        break
    run_dict["best_run_path"] = run.config["config"]["CHECKPOINT_FOLDER"] + f"/ckpt_epoch_{filter_metric_idx}.pth"
    run_dict["best_run_idx"] = filter_metric_idx
        
    run_dict[filter_metric] = run_history[filter_metric][filter_metric_idx]
    
    # add other metrics corresponding to filter_operation filter_metric
    for metric in metrics:
        # catch models that only evaluate some actions
        if metric in run_history.keys():
            run_dict[metric] = run_history[metric][filter_metric_idx]
        else:
            run_dict[metric] = -1
    
    runs_df = runs_df.append(run_dict, ignore_index=True)
    
runs_df = runs_df.sort_values(by=['name'])
runs_df

  0%|                                                                                                                                                                            | 0/85 [00:00<?, ?it/s]

vo_vit_s_scratch_rgb[balmy_moon_71]


 56%|████████████████████████████████████████████████████████████████████████████████████████████                                                                       | 48/85 [00:00<00:00, 84.97it/s]

vo_vit_s_scratch_d[logical_pyramid_66]
vo_vit_s_scratch_act_rgb[graceful_water_65]
vo_vit_s_scratch_act_rgbd[splendid_butterfly_64]
vo_vit_s_scratch_act_d[unique_bee_63]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 85/85 [00:03<00:00, 23.27it/s]


,name,id,state,config.VO.MODEL.visual_backbone,config.VO.MODEL.pretrain_backbone,config.VO.MODEL.train_backbone,config.VO.MODEL.visual_type,config.VO.MODEL.cls_action,Objective/eval_all,Objective/eval_forward,Objective/eval_left,Objective/eval_right,eval_all_unified_depth/aux_depth_loss,Objective/train,best_run_idx,best_run_path
4,vo_vit_s_scratch_act_d,2t4643i0,finished,small,None,True,[depth],True,0.001075,0.000509,0.000508,0.000514,0,0.000066,148.0,train_log/final/vit/unique/vo_vit_s_scratch_ac...
2,vo_vit_s_scratch_act_rgb,ifivs870,finished,small,None,True,[rgb],True,0.016451,0.015648,0.005292,0.005373,0,0.017388,10.0,train_log/final/vit/unique/vo_vit_s_scratch_ac...
3,vo_vit_s_scratch_act_rgbd,2fa17w9h,finished,small,None,True,"[rgb, depth]",True,0.001468,0.000711,0.000668,0.000695,0,0.000145,126.0,train_log/final/vit/unique/vo_vit_s_scratch_ac...
1,vo_vit_s_scratch_d,37ezwpmf,finished,small,None,True,[depth],False,0.002266,0.001066,0.000849,0.001023,0,0.001077,147.0,train_log/final/vit/unique/vo_vit_s_scratch_d/...
0,vo_vit_s_scratch_rgb,gbp3ylyb,finished,small,None,True,[rgb],False,0.006207,0.006740,0.001743,0.001548,0,0.004858,146.0,train_log/final/vit/unique/vo_vit_s_scratch_rg...


In [79]:
dt = datetime.datetime.now().strftime("%Y%m%d")
filename = f"{out_name}_{dt}"
filename

'exp_vo_scratch_20220505'

In [80]:
# save df as excel file
extension = ".xlsx"
with pd.ExcelWriter(filename+extension) as excel_writer:
    runs_df.to_excel(excel_writer, sheet_name='Sheet1')

In [45]:
# # save df as csv file
# extension = ".csv"
# runs_df.to_csv(filename+extension)